# 1. - Bajar información relevante

In [2]:
import sys
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
df_loan_train = pd.read_csv('./data/train_data_processed.csv')
df_loan_test = pd.read_csv('./data/test_data_processed.csv')
df_loan_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,TARGET,AMT_REQ_CREDIT_BUREAU_HOUR_input,AMT_REQ_CREDIT_BUREAU_DAY_input,AMT_REQ_CREDIT_BUREAU_WEEK_input,AMT_REQ_CREDIT_BUREAU_MON_input,AMT_REQ_CREDIT_BUREAU_QRT_input,AMT_REQ_CREDIT_BUREAU_YEAR_input
0,159213,Cash loans,F,N,Y,0.0,67500.0,675000.0,19476.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,151735,Cash loans,M,Y,Y,2.0,315000.0,1288350.0,37800.0,1125000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,381923,Cash loans,F,N,N,1.0,112500.0,450000.0,25834.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,432341,Cash loans,M,Y,Y,0.0,270000.0,1079581.5,52065.0,886500.0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,3.0
4,273574,Cash loans,F,N,N,0.0,90000.0,204858.0,14386.5,171000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0


# 2. - Codificación variables categóricas

La codificación de variables categóricas tiene como propósito transformar datos categóricos en un formato numérico que pueda ser entendido y procesado por algoritmos de machine learning. Dado que la mayoría de los modelos estadísticos y de machine learning no trabajan directamente con datos no numéricos, es necesario convertir las categorías en representaciones matemáticas. La codificación garantiza que estas transformaciones preserven las relaciones y patrones subyacentes en los datos, minimizando la pérdida de información.

Tipos de Codificación
1. **One-Hot Encoding**
    - Consiste en: Crear una nueva columna binaria (0/1) para cada categoría en la variable original. Si una fila pertenece a una categoría, la columna correspondiente toma el valor 1; de lo contrario, toma 0.
    - Se suele aplicar: A variables categóricas nominales (sin orden inherente), como colores, ciudades, o marcas. Es ideal para variables con pocas categorías.
    - Ejemplo: La categoría "Color" con valores {Rojo, Azul, Verde} generaría tres columnas: "Color_Rojo", "Color_Azul", "Color_Verde".
2. **Label Encoding**
    - Consiste en: Asignar un número entero a cada categoría de la variable (e.g., "Rojo" = 0, "Azul" = 1, "Verde" = 2).
    - Se suele aplicar: A variables categóricas ordinales (donde existe un orden lógico entre categorías), como niveles educativos ("Bajo", "Medio", "Alto").
    - Ejemplo: La categoría "Tamaño" con valores {Pequeño, Mediano, Grande} podría codificarse como {0, 1, 2}, respetando el orden inherente.
3. **Binary Encoding**
    - Consiste en: Convertir las categorías en valores binarios y utilizar las posiciones binarias como columnas. Este método reduce el número de columnas en comparación con One-Hot Encoding.
    - Se suele aplicar: A variables con alta cardinalidad (muchas categorías) para reducir la dimensionalidad mientras se preserva información.
    - Ejemplo: Una categoría con valores {1, 2, 3, 4} podría transformarse en sus representaciones binarias: {00, 01, 10, 11}.
4. **Target Encoding (Codificación Basada en el Objetivo)**
    - Consiste en: Sustituir cada categoría por una estadística calculada a partir de la variable objetivo (e.g., media, proporción o mediana de la categoría con respecto al objetivo).
    - Se suele aplicar: En problemas supervisados donde la variable objetivo está disponible, y particularmente útil para variables categóricas con alta cardinalidad.
    - Ejemplo: Si la variable objetivo es binaria (0/1), la categoría "A" podría reemplazarse con la proporción de 1 dentro de esa categoría.
5. **Frequency Encoding**
    - Consiste en: Sustituir cada categoría por la frecuencia relativa o absoluta con la que aparece en la columna.
    - Se suele aplicar: A variables categóricas con muchas categorías, especialmente cuando la frecuencia tiene un significado relevante en el análisis.
    - Ejemplo: Si la categoría "A" aparece en el 50% de las filas, se codificaría con el valor 0.5.
6. **Hashing Encoding**
    - Consiste en: Aplicar una función hash para asignar cada categoría a un espacio reducido de columnas, basado en el resultado del hash. No requiere conocer todas las categorías por adelantado.
    - Se suele aplicar: A datasets muy grandes con variables categóricas de muy alta cardinalidad, o en sistemas en tiempo real donde las categorías no son fijas.
    - Ejemplo: La categoría "Rojo" podría mapearse a un hash único como columna binaria dentro de un espacio reducido.



## 2.1. - Análisis 

In [9]:
lista_variables_codificar = df_loan_train.select_dtypes('object').columns.to_list()
df_loan_train[lista_variables_codificar].nunique()

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               19
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
HOUSETYPE_MODE                 4
WALLSMATERIAL_MODE             8
EMERGENCYSTATE_MODE            3
dtype: int64

In [ ]:
df_loan_train.WEEKDAY_APPR_PROCESS_START.value_counts() 

WEEKDAY_APPR_PROCESS_START
TUESDAY      43245
WEDNESDAY    41607
MONDAY       40656
THURSDAY     40340
FRIDAY       40163
SATURDAY     27115
SUNDAY       12882
Name: count, dtype: int64

He decidido aplicar diferentes tipos de codificación en función de las características específicas de las variables categóricas de mi dataset para maximizar la información que aportan al modelo, manteniendo la simplicidad y evitando introducir relaciones artificiales.

1. One-Hot Encoding: Este método será aplicado a la mayoría de las variables categóricas, ya que presentan un número reducido de categorías y no tienen un orden inherente. Este enfoque es ideal para variables nominales, permitiendo representar cada categoría como una columna independiente sin introducir relaciones ficticias entre ellas. Además, al no tener demasiadas categorías, no se generará un problema significativo de aumento de dimensionalidad.

2. Label Encoding para WEEKDAY_APPR_PROCESS_START: Para esta variable, que representa el día de la semana en el que se inicia el proceso de aprobación, aplicaré Label Encoding. Aunque no es una variable ordinal estricta, existe un orden implícito en los días de la semana que puede ser relevante para el modelo. Este método asignará un valor numérico a cada día (por ejemplo, 0 para lunes, 1 para martes, etc.), respetando el orden natural de esta variable.

3. Target Encoding para ORGANIZATION_TYPE y OCCUPATION_TYPE: Estas variables tienen un alto número de categorías (alta cardinalidad), lo que las hace poco adecuadas para métodos como One-Hot Encoding. Aplicaré Target Encoding, que reemplazará cada categoría con una estadística calculada a partir de la variable objetivo, como la media de TARGET para cada categoría. Este enfoque permite conservar la relación entre estas categorías y la variable objetivo, optimizando su utilidad en modelos supervisados.


## 2.2. - Proceso de codificación

In [ ]:
X_train = df_loan_train.drop('TARGET',axis=1)
X_test = df_loan_test.drop('TARGET',axis=1)
y_train = df_loan_train['TARGET']
y_test = df_loan_test['TARGET']

In [14]:
## LABEL ENCODING 

from sklearn.preprocessing import LabelEncoder

# Crear el codificador
label_encoder = LabelEncoder()

# Aplicar Label Encoding en WEEKDAY_APPR_PROCESS_START en el conjunto de entrenamiento
X_train['WEEKDAY_APPR_PROCESS_START'] = label_encoder.fit_transform(X_train['WEEKDAY_APPR_PROCESS_START'])

# Aplicar el mismo mapeo en el conjunto de prueba
X_test['WEEKDAY_APPR_PROCESS_START'] = label_encoder.transform(X_test['WEEKDAY_APPR_PROCESS_START'])

# Verificar el resultado
print("Valores únicos tras codificación en entrenamiento:", X_train['WEEKDAY_APPR_PROCESS_START'].unique())
print("Valores únicos tras codificación en prueba:", X_test['WEEKDAY_APPR_PROCESS_START'].unique())


Valores únicos tras codificación en entrenamiento: [1 0 5 4 6 2 3]
Valores únicos tras codificación en prueba: [0 3 1 4 5 6 2]


In [ ]:
### TARGET ENCODING
import category_encoders as ce

# Variables a codificar
target_columns = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']

# Crear y ajustar el codificador de Target Encoding
target_enc = ce.TargetEncoder(cols=target_columns)
target_enc.fit(X_train[target_columns], y_train)  # Ajustar usando el conjunto de entrenamiento

# Transformar directamente X_train y X_test
X_train[target_columns] = target_enc.transform(X_train[target_columns])
X_test[target_columns] = target_enc.transform(X_test[target_columns])

# Verificar formas finales y primeras filas
print("Formas finales:", X_train.shape, X_test.shape)
print("Primeras filas de entrenamiento:\n", X_train[target_columns].head())
print("Primeras filas de prueba:\n", X_test[target_columns].head())


lista_variables_codificar.remove('ORGANIZATION_TYPE')
lista_variables_codificar.remove('OCCUPATION_TYPE')
lista_variables_codificar.remove('WEEKDAY_APPR_PROCESS_START')

Formas finales: (246008, 105) (61503, 105)
Primeras filas de entrenamiento:
    ORGANIZATION_TYPE  OCCUPATION_TYPE
0           0.101976         0.096905
1           0.065301         0.062554
2           0.092402         0.062554
3           0.084058         0.062075
4           0.054438         0.064856
Primeras filas de prueba:
    ORGANIZATION_TYPE  OCCUPATION_TYPE
0           0.054438         0.064856
1           0.079245         0.105153
2           0.093034         0.064856
3           0.069612         0.105153
4           0.054438         0.064856


In [18]:
### One-hot encding

# Crear y aplicar One-Hot Encoder
ohe = ce.OneHotEncoder(cols=lista_variables_codificar, use_cat_names=True)
ohe.fit(X_train, y_train)  

# Transformar X_train y X_test
X_train = ohe.transform(X_train)
X_test = ohe.transform(X_test)

# Verificar formas finales
print(X_train.shape, X_test.shape)

(246008, 149) (61503, 149)


Después de realizar las transformaciones y codificaciones en el dataset, la dimensionalidad ha aumentado de 106 variables iniciales a 149 variables. Este incremento se debe principalmente a la aplicación de técnicas como One-Hot Encoding y Target Encoding, que han generado nuevas columnas para representar las categorías originales de manera numérica. Aunque el aumento en el número de variables puede incrementar la complejidad del modelo, estas transformaciones son esenciales para garantizar que la información categórica sea correctamente interpretada por los algoritmos de machine learning.



# 3. - Escalado de variables

El escalado de variables tiene como objetivo normalizar o estandarizar las características del dataset para garantizar que todas tengan un rango similar o comparable. Esto es especialmente importante en algoritmos que dependen de la magnitud de las variables o que calculan distancias, como regresión logística, SVM, k-NN, y redes neuronales. Variables con diferentes escalas (por ejemplo, ingresos en miles frente a edad en años) pueden causar que el modelo otorgue más importancia a unas características simplemente por su magnitud.

Métodos de Escalado de Variables

1. Min-Max Scaling (Normalización):
    - Escala las variables para que sus valores estén dentro de un rango definido, generalmente entre 0 y 1.
    - Resta el valor mínimo de la variable y lo divide por el rango total (máximo - mínimo).
    - Datos con distribuciones conocidas y sin outliers.

2. StandardScaler (Estandarización):
    - Ajusta las variables para que tengan media 0 y desviación estándar 1.
    - Resta la media y divide entre la desviación estándar.
    - Modelos sensibles a magnitudes o rangos, como SVM, regresión logística o PCA.

3. RobustScaler:
    - Escala las variables usando los cuartiles, siendo robusto frente a valores atípicos.
    - ustituye la media por la mediana y divide por el rango intercuartílico (IQR).
    - Datos con outliers extremos que podrían distorsionar otros métodos.

4. MaxAbsScaler:
    - Escala las variables dividiendo por el valor absoluto máximo, ajustando el rango entre -1 y 1.
    - Cada valor se divide por el valor absoluto más grande en su columna.
    - Datos dispersos o representados como matrices esparsas.

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
model_scaled = scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [20]:
X_train_scaled.describe()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,FLAG_OWN_REALTY_N,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR_input,AMT_REQ_CREDIT_BUREAU_DAY_input,AMT_REQ_CREDIT_BUREAU_WEEK_input,AMT_REQ_CREDIT_BUREAU_MON_input,AMT_REQ_CREDIT_BUREAU_QRT_input,AMT_REQ_CREDIT_BUREAU_YEAR_input
count,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,...,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05
mean,2.014330e-16,-2.888291e-19,2.888291e-19,-2.812474e-18,3.813988e-17,8.033060e-19,-7.079202e-17,7.079202e-17,-5.155600e-18,5.155600e-18,...,-2.385729e-17,8.780405e-18,1.379159e-18,-6.910237e-18,-3.364859e-18,2.327963e-17,-5.611950e-17,4.332437e-17,-5.653830e-18,6.206938e-17
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-1.733579e+00,-3.080923e+00,-3.245780e-01,-1.386971e+00,-7.209694e-01,-4.032356e-03,-1.395266e+00,-7.167090e-01,-1.505871e+00,-6.640677e-01,...,-9.055708e-02,-2.386234e-02,-2.218324e-02,-1.859131e-02,-8.251899e-02,-6.683547e-02,-1.780977e-01,-3.124185e-01,-4.651899e-01,-1.093201e+00
25%,-8.673068e-01,3.245780e-01,-3.245780e-01,-1.386971e+00,-7.209694e-01,-4.032356e-03,-1.395266e+00,-7.167090e-01,-1.505871e+00,-6.640677e-01,...,-9.055708e-02,-2.386234e-02,-2.218324e-02,-1.859131e-02,-8.251899e-02,-6.683547e-02,-1.780977e-01,-3.124185e-01,-4.651899e-01,-5.179792e-01
50%,2.582610e-03,3.245780e-01,-3.245780e-01,7.209954e-01,-7.209694e-01,-4.032356e-03,7.167090e-01,-7.167090e-01,6.640677e-01,-6.640677e-01,...,-9.055708e-02,-2.386234e-02,-2.218324e-02,-1.859131e-02,-8.251899e-02,-6.683547e-02,-1.780977e-01,-3.124185e-01,-4.651899e-01,-3.670641e-03
75%,8.656491e-01,3.245780e-01,-3.245780e-01,7.209954e-01,1.387021e+00,-4.032356e-03,7.167090e-01,1.395266e+00,6.640677e-01,1.505871e+00,...,-9.055708e-02,-2.386234e-02,-2.218324e-02,-1.859131e-02,-8.251899e-02,-6.683547e-02,-1.780977e-01,-4.250889e-03,8.276692e-03,6.324642e-01
max,1.730169e+00,3.245780e-01,3.080923e+00,7.209954e-01,1.387021e+00,2.479940e+02,7.167090e-01,1.395266e+00,6.640677e-01,1.505871e+00,...,1.104276e+01,4.190704e+01,4.507909e+01,5.378858e+01,5.095343e+01,8.840553e+01,4.212354e+01,3.139778e+01,3.281935e+01,1.328734e+01
